# Passo 2: Análise Exploratória Geral (Descriptive Analytics)

## Objetivo
Este notebook tem como foco a análise descritiva dos dados de varejo. Antes de entrarmos em análises complexas de elasticidade e causalidade, precisamos entender:
1.  **Quem são os campeões de vendas?** (Pareto: Receita e Volume)
2.  **Como as vendas se comportam ao longo do tempo?** (Sazonalidade e Tendência Geral)
3.  **Qual a distribuição de preços e categorias?**
4.  **Qualidade dos dados** (visão geral das distribuições).

In [0]:

# Cores padrão para gráficos
colors_default = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')


## 1. Carregamento e Visão Geral

In [0]:
# [AUTO-UPDATED] Load from Spark Table
from pyspark.sql import SparkSession
import pandas as pd
import os

# Initialize Spark
spark = SparkSession.builder.appName("EDA").config("spark.sql.warehouse.dir", "spark-warehouse").getOrCreate()
# Carregar dados
query = 'SELECT * FROM elastic_price.raw.retail_price'
df = spark.sql(query)
# Mostrar dados
display(df)

## 2. Top Produtos (Best Sellers)
Quem gera mais receita? Quem vende mais unidades?

In [0]:
# Top 10 Produtos por Receita
query = '''
SELECT product_id, product_category_name, SUM(total_revenue) as total_revenue
FROM elastic_price.raw.retail_price
GROUP BY product_id, product_category_name
ORDER BY total_revenue DESC
LIMIT 10
'''
df_rev = pd.DataFrame(spark.sql(query).collect(), columns=spark.sql(query).columns)

# Criar o gráfico
plt.figure(figsize=(14, 6))
plt.bar(df_rev['product_id'], df_rev['total_revenue'], color=colors_default[0])
plt.title('Top 10 Produtos por Receita Total')
plt.xlabel('Produto')
plt.ylabel('Receita ($)')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [0]:
# Top 10 Produtos por Volume
query = '''
SELECT product_id, product_category_name, SUM(qty) as total_qty
FROM elastic_price.raw.retail_price
GROUP BY product_id, product_category_name
ORDER BY total_qty DESC
LIMIT 10
'''
df_qty = pd.DataFrame(spark.sql(query).collect(), columns=spark.sql(query).columns)

# Criar o gráfico
plt.figure(figsize=(14, 6))
plt.bar(df_qty['product_id'], df_qty['total_qty'], color=colors_default[1])
plt.title('Top 10 Produtos por Volume de Vendas (Qtd)')
plt.xlabel('Produto')
plt.ylabel('Quantidade Vendida')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 3. Histórico de Vendas (Evolução Temporal)
Como a receita total da empresa evoluiu mês a mês?

In [0]:
# Evolução da Receita Mensal Global
query = '''
SELECT DATE_FORMAT(date, 'yyyy-MM-01') AS ANO_MES, 
       SUM(total_revenue) AS VALOR_VENDA_LIQUIDA
FROM elastic_price.raw.retail_price
GROUP BY DATE_FORMAT(date, 'yyyy-MM-01')
ORDER BY ANO_MES
'''
df = pd.DataFrame(spark.sql(query).collect(), columns=spark.sql(query).columns).set_index('ANO_MES').astype(float)

# Criar o gráfico de linha usando matplotlib
plt.figure(figsize=(14, 4))
df.VALOR_VENDA_LIQUIDA.plot(color=colors_default[7], marker='o')
plt.title('Evolução da Venda Líquida Global')
plt.xlabel('Mês')
plt.ylabel('Valor da Venda Líquida (R$)')
plt.grid(True)
plt.tight_layout()
plt.show()

### 3.1 Vazamento de Tendência por Categoria
Quais categorias estão crescendo?

In [0]:
# Evolução da Receita por Categoria
query = '''
SELECT DATE_FORMAT(date, 'yyyy-MM-01') AS ANO_MES, 
       product_category_name,
       SUM(total_revenue) AS total_revenue
FROM elastic_price.raw.retail_price
GROUP BY DATE_FORMAT(date, 'yyyy-MM-01'), product_category_name
ORDER BY ANO_MES
'''
df_cat = pd.DataFrame(spark.sql(query).collect(), columns=spark.sql(query).columns)
df_cat['total_revenue'] = df_cat['total_revenue'].astype(float)
df_pivot = df_cat.pivot(index='ANO_MES', columns='product_category_name', values='total_revenue').fillna(0)

plt.figure(figsize=(14, 6))
df_pivot.plot(kind='area', stacked=True, linewidth=0, ax=plt.gca())
plt.title('Evolução da Receita por Categoria')
plt.xlabel('Mês')
plt.ylabel('Receita ($)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Análise de Preços e Categorias
Distribuição dos preços praticados.

In [0]:
# Distribuição de Preços Unitários
query = '''
SELECT unit_price
FROM elastic_price.raw.retail_price
'''
df_price = pd.DataFrame(spark.sql(query).collect(), columns=spark.sql(query).columns).astype(float)

plt.figure(figsize=(10, 6))
plt.hist(df_price['unit_price'], bins=50, color='teal', edgecolor='black', alpha=0.7)
plt.title('Distribuição de Preços Unitários')
plt.xlabel('Preço ($)')
plt.ylabel('Frequência')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [0]:
# Boxplot de Preço por Categoria
query = '''
SELECT product_category_name, unit_price
FROM elastic_price.raw.retail_price
'''
df_box = pd.DataFrame(spark.sql(query).collect(), columns=spark.sql(query).columns)
df_box['unit_price'] = df_box['unit_price'].astype(float)

plt.figure(figsize=(12, 6))
sns.boxplot(data=df_box, x='product_category_name', y='unit_price', palette='viridis')
plt.title('Dispersão de Preços por Categoria')
plt.xticks(rotation=45)
plt.yscale('log')
plt.ylabel('Preço Unitário (Log Scale)')
plt.grid(True, axis='y', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Conclusão da Análise Descritiva
Esta visão geral nos permite identificar:
- Os produtos "Geração de Caixa" (Top Sellers).
- A sazonalidade do negócio.
- As categorias com maior ticket médio.

**Próximo Passo:** No notebook `03_eda_elasticity`, aprofundaremos na relação Preço x Demanda (Elasticidade) e Decomposição de Séries Temporais para o produto principal.